In [1]:
import pandas as pd
import numpy as np
from collections import defaultdict

In [2]:
pd.options.display.max_rows = 100

In [3]:
!pwd

/Users/malte.luecken/helmholtz_munich/benchmarking_data_integration/scib-reproducibility/notebooks/analysis


In [4]:
files = !ls ../../../Paper/202109_kBET_fix/Supplementary\ Files/Results/*.csv

In [5]:
data = {file:pd.read_csv(file) for file in files}

In [6]:
data[files[0]]

,Unnamed: 0,Method,Output,Features,Scaling,Overall Score,Batch Correction,PCR batch,Batch ASW,graph iLISI,...,Bio conservation,NMI cluster/label,ARI cluster/label,Cell type ASW,isolated label F1,isolated label silhouette,graph cLISI,CC conservation,HVG conservation,trajectory conservation
0,319,scGen*,gene,HVG,unscaled,0.667406,0.619712,0.593345,0.841671,0.106118,...,0.699201,0.904241,0.849627,0.675813,0.155460,0.391633,1.000000,0.491212,0.416649,0.925860
1,43,Scanorama,embed,HVG,scaled,0.641593,0.677942,0.901327,0.955880,0.107809,...,0.617360,0.637543,0.540902,0.501580,0.159606,0.438662,0.967787,0.706393,NaN,0.888212
2,235,scVI,embed,HVG,unscaled,0.636712,0.620568,0.870668,0.917481,0.078631,...,0.647474,0.687811,0.544036,0.518036,0.183759,0.457020,0.988365,0.569934,NaN,0.934886
3,247,scANVI*,embed,HVG,unscaled,0.627515,0.589593,0.799600,0.891219,0.068181,...,0.652797,0.746849,0.612477,0.569162,0.076483,0.441384,0.996387,0.652344,NaN,0.905722
4,320,scGen*,gene,FULL,unscaled,0.625399,0.605967,0.594643,0.829299,0.090718,...,0.638353,0.900034,0.849291,0.665508,0.155627,0.417719,1.000000,0.413542,0.237884,0.661730
5,7,MNN,gene,HVG,unscaled,0.620253,0.549339,0.623340,0.923168,0.053692,...,0.667529,0.673820,0.527823,0.526868,0.243669,0.485912,0.992262,0.782074,0.342327,0.863365
6,236,scVI,embed,FULL,unscaled,0.609465,0.579523,0.795729,0.896165,0.062366,...,0.629426,0.663141,0.421125,0.514007,0.240570,0.433753,0.992627,0.555435,NaN,0.938736
7,44,Scanorama,embed,FULL,scaled,0.606155,0.629816,0.904613,0.940262,0.074805,...,0.590382,0.652132,0.476423,0.489695,0.140711,0.413106,0.987966,0.659911,NaN,0.913838
8,343,fastMNN,embed,HVG,unscaled,0.604928,0.558267,0.589596,0.910170,0.065643,...,0.636036,0.666366,0.538022,0.524583,0.086478,0.472398,0.989702,0.704683,NaN,0.905399
9,104,Seurat v3 RPCA,gene,FULL,unscaled,0.600383,0.626742,0.931048,0.873890,0.117627,...,0.582810,0.554957,0.333970,0.502028,0.337565,0.470623,0.980930,0.679879,0.255777,0.721897


# Check min-max-scaled score contributions for each method

In [7]:
batch_scores = ['PCR batch', 'Batch ASW', 'graph iLISI', 'graph connectivity', 'kBET']
bio_scores = ['NMI cluster/label', 'ARI cluster/label', 'Cell type ASW', 'isolated label F1', 'isolated label silhouette', 'graph cLISI', 'CC conservation', 'HVG conservation', 'trajectory conservation']

In [8]:
def max_min_scale_cols(df):
    return((df - df.min())/(df.max() - df.min()))

In [9]:
data[files[0]].columns

Index(['Unnamed: 0', 'Method', 'Output', 'Features', 'Scaling',
       'Overall Score', 'Batch Correction', 'PCR batch', 'Batch ASW',
       'graph iLISI', 'graph connectivity', 'kBET', 'Bio conservation',
       'NMI cluster/label', 'ARI cluster/label', 'Cell type ASW',
       'isolated label F1', 'isolated label silhouette', 'graph cLISI',
       'CC conservation', 'HVG conservation', 'trajectory conservation'],
      dtype='object')

In [10]:
# Generate metric result ranks per method
data_mm_scaled = dict()

for file in data:
    data_mm_scaled[file] = max_min_scale_cols(data[file][[col for col in batch_scores+bio_scores if col in data[file].columns]])

    #Save  method ID output
    data_mm_scaled[file]['method_id'] = ['_'.join(data[file][['Method', 'Output', 'Features', 'Scaling']].values[i]) for i in range(data[file].shape[0])]
    data_mm_scaled[file]['Bio conservation'] = data[file]['Bio conservation']
    data_mm_scaled[file]['Batch Correction'] = data[file]['Batch Correction']
    data_mm_scaled[file]['Overall Score'] = data[file]['Overall Score']

    data_mm_scaled[file] = data_mm_scaled[file][~data_mm_scaled[file]['method_id'].isin(['Unintegrated_gene_FULL_unscaled'])]
    
    for col in batch_scores+bio_scores:
        if col not in data[file].columns:
            continue
        else:
            data_mm_scaled[file][col+'_rank'] = data_mm_scaled[file][col].rank(ascending=False)
        

In [11]:
# Average rank per method across tasks
merged = pd.concat(data_mm_scaled.values())

In [12]:
mean_ranks_per_method = merged.groupby(['method_id']).agg({col+'_rank':np.nanmean for col in batch_scores+bio_scores})

In [13]:
mean_ranks_per_method.loc[['Scanorama_embed_HVG_scaled', 'scANVI*_embed_HVG_unscaled', 'scGen*_gene_HVG_unscaled', 'fastMNN_embed_HVG_unscaled', 
                           'Harmony_embed_HVG_unscaled', 'Seurat v3 CCA_gene_HVG_unscaled', 'scVI_embed_HVG_unscaled', 'Seurat v3 RPCA_gene_HVG_scaled', 
                          'trVAE_embed_HVG_unscaled', 'BBKNN_graph_HVG_unscaled']]

,PCR batch_rank,Batch ASW_rank,graph iLISI_rank,graph connectivity_rank,kBET_rank,NMI cluster/label_rank,ARI cluster/label_rank,Cell type ASW_rank,isolated label F1_rank,isolated label silhouette_rank,graph cLISI_rank,CC conservation_rank,HVG conservation_rank,trajectory conservation_rank
method_id,,,,,,,,,,,,,,
Scanorama_embed_HVG_scaled,22.285714,2.571429,19.285714,22.285714,21.000000,14.500000,14.071429,33.285714,17.214286,35.571429,35.428571,21.800000,NaN,18.0
scANVI*_embed_HVG_unscaled,28.000000,23.428571,34.142857,10.571429,32.166667,5.785714,6.357143,10.428571,21.500000,15.857143,11.928571,30.600000,NaN,22.0
scGen*_gene_HVG_unscaled,42.166667,37.833333,28.833333,8.833333,15.166667,2.833333,2.666667,4.500000,13.666667,33.500000,9.166667,33.500000,5.0,4.5
fastMNN_embed_HVG_unscaled,36.000000,16.857143,34.428571,24.357143,26.166667,12.928571,13.357143,14.000000,22.785714,28.857143,18.000000,22.800000,NaN,23.0
Harmony_embed_HVG_unscaled,30.000000,20.428571,24.285714,33.714286,29.833333,23.714286,21.285714,20.428571,33.285714,27.285714,22.928571,18.400000,NaN,17.5
Seurat v3 CCA_gene_HVG_unscaled,23.200000,37.400000,18.600000,32.800000,39.000000,29.600000,26.600000,28.400000,38.800000,25.600000,29.500000,39.666667,3.0,56.0
scVI_embed_HVG_unscaled,19.428571,21.142857,30.571429,16.285714,35.500000,14.428571,13.142857,24.857143,24.285714,18.571429,26.857143,36.600000,NaN,22.5
Seurat v3 RPCA_gene_HVG_scaled,31.400000,34.600000,29.800000,27.200000,23.000000,26.400000,28.400000,18.800000,28.200000,27.600000,25.700000,29.250000,11.8,37.5
trVAE_embed_HVG_unscaled,27.800000,18.400000,33.600000,27.100000,26.600000,39.400000,36.400000,49.400000,27.400000,24.400000,48.200000,47.333333,NaN,42.0


In [14]:
mean_ranks_per_method.loc[[idx for idx in mean_ranks_per_method.index if idx.startswith('Scanorama')]]

,PCR batch_rank,Batch ASW_rank,graph iLISI_rank,graph connectivity_rank,kBET_rank,NMI cluster/label_rank,ARI cluster/label_rank,Cell type ASW_rank,isolated label F1_rank,isolated label silhouette_rank,graph cLISI_rank,CC conservation_rank,HVG conservation_rank,trajectory conservation_rank
method_id,,,,,,,,,,,,,,
Scanorama_embed_FULL_scaled,26.500000,2.500000,33.666667,25.000000,35.000000,17.583333,22.083333,36.666667,18.750000,28.666667,33.916667,31.0,NaN,22.0
Scanorama_embed_FULL_unscaled,47.428571,20.428571,53.714286,39.142857,53.083333,23.714286,30.285714,36.000000,22.214286,18.142857,16.785714,17.6,NaN,3.0
Scanorama_embed_HVG_scaled,22.285714,2.571429,19.285714,22.285714,21.000000,14.500000,14.071429,33.285714,17.214286,35.571429,35.428571,21.8,NaN,18.0
Scanorama_embed_HVG_unscaled,43.428571,13.000000,48.571429,39.714286,48.750000,13.500000,18.642857,25.714286,24.285714,30.857143,14.000000,23.8,NaN,12.0
Scanorama_gene_FULL_scaled,41.166667,39.500000,38.333333,43.666667,50.333333,29.083333,28.750000,27.166667,19.166667,18.000000,27.666667,21.0,25.333333,56.0
Scanorama_gene_FULL_unscaled,51.000000,32.000000,52.785714,42.142857,52.250000,26.428571,34.285714,30.714286,19.142857,17.142857,14.214286,11.2,15.500000,11.5
Scanorama_gene_HVG_scaled,30.142857,36.714286,25.285714,40.857143,47.666667,19.714286,17.857143,20.142857,19.714286,18.571429,28.428571,23.2,11.857143,24.0
Scanorama_gene_HVG_unscaled,45.000000,33.285714,44.714286,42.071429,54.250000,20.000000,25.428571,17.142857,18.071429,29.142857,12.428571,21.0,4.857143,12.5


In [15]:
metric_scaling_effects = pd.DataFrame(index=mean_ranks_per_method.columns, columns=['UH_unscaled_prop', 'UH_unscaled_num', 'Total_UH_vals', 'UH_metric_effect', 'UQ_unscaled_prop', 'UQ_unscaled_num', 'Total_UQ_vals', 'UQ_metric_effect'])
# Note: UH = upper half; UQ = upper quarter

for col in mean_ranks_per_method:
    sorted_vals = mean_ranks_per_method[col].sort_values()
    sorted_vals = sorted_vals[~np.isnan(sorted_vals)]
    sorted_idx = sorted_vals.index
    
    upper_half = round(len(sorted_idx)/2)
    upper_quarter = round(len(sorted_idx)/4)
    
    res = [True if idx.endswith('unscaled') else False for idx in sorted_idx[:upper_half]]
    metric_scaling_effects['UH_unscaled_prop'][col] = np.mean(res)
    metric_scaling_effects['UH_unscaled_num'][col] = np.sum(res)
    metric_scaling_effects['Total_UH_vals'][col] = len(res)
    metric_scaling_effects['UH_metric_effect'][col] = np.abs(np.mean(res) - 0.5)

    res = [True if idx.endswith('unscaled') else False for idx in sorted_idx[:upper_quarter]]
    metric_scaling_effects['UQ_unscaled_prop'][col] = np.mean(res)
    metric_scaling_effects['UQ_unscaled_num'][col] = np.sum(res)
    metric_scaling_effects['Total_UQ_vals'][col] = len(res)
    metric_scaling_effects['UQ_metric_effect'][col] = np.abs(np.mean(res) - 0.5)

In [16]:
metric_scaling_effects.sort_values(by='UH_metric_effect', ascending=False)
metric_scaling_effects.sort_values(by='UQ_metric_effect', ascending=False)

,UH_unscaled_prop,UH_unscaled_num,Total_UH_vals,UH_metric_effect,UQ_unscaled_prop,UQ_unscaled_num,Total_UQ_vals,UQ_metric_effect
trajectory conservation_rank,0.735294,25,34,0.235294,0.882353,15,17,0.382353
graph cLISI_rank,0.705882,24,34,0.205882,0.882353,15,17,0.382353
isolated label silhouette_rank,0.7,21,30,0.2,0.733333,11,15,0.233333
HVG conservation_rank,0.6875,11,16,0.1875,0.875,7,8,0.375
NMI cluster/label_rank,0.676471,23,34,0.176471,0.764706,13,17,0.264706
ARI cluster/label_rank,0.676471,23,34,0.176471,0.823529,14,17,0.323529
PCR batch_rank,0.333333,11,33,0.166667,0.125,2,16,0.375
isolated label F1_rank,0.647059,22,34,0.147059,0.529412,9,17,0.0294118
Cell type ASW_rank,0.633333,19,30,0.133333,0.666667,10,15,0.166667
graph connectivity_rank,0.617647,21,34,0.117647,0.588235,10,17,0.0882353


,UH_unscaled_prop,UH_unscaled_num,Total_UH_vals,UH_metric_effect,UQ_unscaled_prop,UQ_unscaled_num,Total_UQ_vals,UQ_metric_effect
graph cLISI_rank,0.705882,24,34,0.205882,0.882353,15,17,0.382353
trajectory conservation_rank,0.735294,25,34,0.235294,0.882353,15,17,0.382353
PCR batch_rank,0.333333,11,33,0.166667,0.125,2,16,0.375
HVG conservation_rank,0.6875,11,16,0.1875,0.875,7,8,0.375
kBET_rank,0.470588,16,34,0.0294118,0.176471,3,17,0.323529
ARI cluster/label_rank,0.676471,23,34,0.176471,0.823529,14,17,0.323529
CC conservation_rank,0.606061,20,33,0.106061,0.8125,13,16,0.3125
NMI cluster/label_rank,0.676471,23,34,0.176471,0.764706,13,17,0.264706
isolated label silhouette_rank,0.7,21,30,0.2,0.733333,11,15,0.233333
graph iLISI_rank,0.411765,14,34,0.0882353,0.294118,5,17,0.205882


In [17]:
mean_ranks_per_method['kBET_rank'].sort_values()

method_id
scGen*_gene_HVG_scaled                5.833333
Harmony_embed_HVG_scaled              7.500000
fastMNN_embed_HVG_scaled             10.666667
fastMNN_gene_HVG_scaled              11.000000
Harmony_embed_FULL_scaled            13.333333
DESC_embed_HVG_scaled                13.500000
scGen*_gene_FULL_unscaled            13.666667
DESC_embed_HVG_unscaled              14.333333
scGen*_gene_HVG_unscaled             15.166667
Conos_graph_HVG_scaled               15.833333
fastMNN_embed_FULL_scaled            16.166667
scGen*_gene_FULL_scaled              16.200000
fastMNN_gene_FULL_scaled             16.666667
BBKNN_graph_FULL_scaled              18.833333
Conos_graph_FULL_scaled              19.166667
BBKNN_graph_HVG_scaled               19.666667
Scanorama_embed_HVG_scaled           21.000000
Seurat v3 RPCA_gene_HVG_scaled       23.000000
LIGER_embed_HVG_unscaled             23.500000
DESC_embed_FULL_unscaled             24.916667
fastMNN_gene_HVG_unscaled            25.333333
fas

# Check Batch vs bio correction score balance per method

In [18]:
top_methods = ['scANVI*_embed_HVG_unscaled', 'Scanorama_embed_HVG_scaled', 'scVI_embed_HVG_unscaled', 'fastMNN_embed_HVG_unscaled', 'scGen*_gene_HVG_unscaled',  'Harmony_embed_HVG_unscaled', 
               'fastMNN_gene_HVG_unscaled', 'Seurat v3 RPCA_gene_HVG_scaled', 'BBKNN_graph_HVG_unscaled', 'Scanorama_gene_HVG_scaled', 'ComBat_gene_HVG_unscaled', 'MNN_gene_HVG_scaled', 
               'Seurat v3 CCA_gene_HVG_unscaled', 'trVAE_embed_HVG_unscaled', 'Conos_graph_FULL_unscaled', 'DESC_embed_FULL_scaled', 'LIGER_embed_HVG_unscaled', 'SAUCIE_embed_HVG_scaled', 'SAUCIE_gene_HVG_scaled', ]

In [19]:
thresh = 0.05

batch_v_bio_top_meth = pd.DataFrame(index=top_methods, columns=['batch_prio', 'balance', 'bio_prio'], data=np.zeros((len(top_methods),3)))

for file in data_mm_scaled:
    if 'simulations' in file:
        continue
    
    data_mm_scaled[file].index = data_mm_scaled[file]['method_id']

    for i in top_methods:
        if i not in data_mm_scaled[file].index:
            continue
            
        diff = data_mm_scaled[file]['Bio conservation'][i] - data_mm_scaled[file]['Batch Correction'][i]
        if diff < -thresh:
            batch_v_bio_top_meth['batch_prio'][i]+=1
        elif -thresh <= diff < thresh:
            batch_v_bio_top_meth['balance'][i]+=1
        else:
            batch_v_bio_top_meth['bio_prio'][i]+=1

In [20]:
batch_v_bio_top_meth.sort_values(by='batch_prio', ascending=False)

,batch_prio,balance,bio_prio
SAUCIE_gene_HVG_scaled,5.0,0.0,0.0
SAUCIE_embed_HVG_scaled,5.0,0.0,0.0
LIGER_embed_HVG_unscaled,5.0,0.0,0.0
BBKNN_graph_HVG_unscaled,5.0,0.0,0.0
Harmony_embed_HVG_unscaled,3.0,0.0,2.0
Seurat v3 RPCA_gene_HVG_scaled,3.0,1.0,0.0
Scanorama_embed_HVG_scaled,3.0,2.0,0.0
Seurat v3 CCA_gene_HVG_unscaled,3.0,0.0,1.0
fastMNN_embed_HVG_unscaled,2.0,0.0,3.0
fastMNN_gene_HVG_unscaled,2.0,3.0,0.0


In [21]:
batch_v_bio_top_meth.sort_values(by='batch_prio', ascending=False).to_csv('Batch_vs_bio_priority_per_RNA_task.csv')

Batch prio:
- SAUCIE
- LIGER
- BBKNN
- Seurat

Balance:
- scVI
- (fastMNN gene)
- Scanorama

Bio prio:
- scANVI
- scGen
- DESC
- Conos